In [1]:
import uuid
from text_process_tools import *  # 假设text_process_tools.py在同一目录下

In [2]:
# 问题一：翻译一个非英语的句子
sentence1 = Sentence(content=[uuid4()], language="fr", sentiment="neu")
# save_object_to_json(sentence1)

language = detect_language_tool(sentence1.id)
translated_sentence1_id = translate_tool(sentence1.id, "en")
senti = sentiment_analysis_sentence_tool(translated_sentence1_id)
print(senti)

neu


In [21]:
# 问题二：将一篇文章从中文翻译为英文，法文，德文,不会超过最大长度限制的版本
Chaps = []
for i in range(3):
    Chap_sentences = []
    for i in range(3):
        s = Sentence(content=[uuid4()], language="zh")
        Chap_sentences.append(s)
    chap = Chapter(sentences=Chap_sentences, language="zh")
    Chaps.append(chap)
article = Article(chapters=Chaps, language="zh")

article_en_id = translate_tool(article.id, "en")

In [19]:
# 问题三：将一篇文章从中文翻译为英文，法文，德文,会超过最大长度限制的版本
Chaps = []
for i in range(3):
    Chap_sentences = []
    for i in range(30):
        s = Sentence(content=[uuid4()], language="zh")
        Chap_sentences.append(s)
    chap = Chapter(sentences=Chap_sentences, language="zh")
    Chaps.append(chap)
article = Article(chapters=Chaps, language="zh")
print(article.word_count)

# 解决方案

chaps_id = split_article_tool(article.id)
print(chaps_id)
T_chaps_id = []
for chap_id in chaps_id:
    translated_chap_id = translate_tool(chap_id, "fr") # 涉及到三个语言
    T_chaps_id.append(translated_chap_id)
translated_article_id = construct_article_tool(chapters_id=T_chaps_id)
print(detect_language_tool(translated_article_id))
translated_article = load_object_from_json(translated_article_id)
print(set(translated_article.content)==set(article.content))

1346
[UUID('133ddec8-dd02-4b83-a5e3-93eb26af5432'), UUID('be9cca67-90c5-4ac7-aa3a-e5c7e7c1dd65'), UUID('121d0449-c89a-46a8-b51d-34f1a18fd7b2')]
fr
True


In [26]:
import uuid
import json
import datetime
from uuid import UUID, uuid4
from typing import List, Optional, Dict, Any
from pydantic import BaseModel, Field, field_validator, model_validator
import random
from pathlib import Path

# Define base models for the Airbnb-like rental system
class Review(BaseModel):
    id: UUID = Field(default_factory=uuid4)
    property_id: int
    user_name: str
    comment: str
    rating: float
    date: str

class Property(BaseModel):
    id: int
    name: str
    address: str
    title: str
    property_type: str
    person_capacity: int
    rating: float
    latitude: float
    longitude: float
    price_per_night: float
    place: str
    
    def model_post_init(self, __context):
        # Auto-save object after initialization
        save_object_to_json(self)

class Availability(BaseModel):
    property_id: int
    dates: List[str]  # List of available dates in YYYY-MM-DD format
    
    def model_post_init(self, __context):
        # Auto-save object after initialization
        save_object_to_json(self)

# Directory to store json files
DATA_DIR = "./airbnb_data"

def save_object_to_json(obj: BaseModel):
    """
    Save the given object as a JSON file.
    """
    # Ensure directory exists
    Path(DATA_DIR).mkdir(parents=True, exist_ok=True)
    
    # Use object's id as filename
    if isinstance(obj, Property):
        file_path = Path(DATA_DIR) / f"property_{obj.id}.json"
    elif isinstance(obj, Review):
        file_path = Path(DATA_DIR) / f"review_{obj.id}.json"
    elif isinstance(obj, Availability):
        file_path = Path(DATA_DIR) / f"availability_{obj.property_id}.json"
    else:
        file_path = Path(DATA_DIR) / f"{obj.id}.json"
    
    # Serialize to JSON string
    json_str = obj.model_dump_json()
    
    # Write to file
    with open(file_path, 'w') as file:
        file.write(json_str)

def load_property_from_json(property_id: int):
    """
    Load a property from JSON file.
    """
    # Ensure directory exists
    Path(DATA_DIR).mkdir(parents=True, exist_ok=True)
    
    file_path = Path(DATA_DIR) / f"property_{property_id}.json"
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
        return Property.model_validate(data)
    except FileNotFoundError:
        return None

def load_availability_from_json(property_id: int):
    """
    Load availability from JSON file.
    """
    # Ensure directory exists
    Path(DATA_DIR).mkdir(parents=True, exist_ok=True)
    
    file_path = Path(DATA_DIR) / f"availability_{property_id}.json"
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
        return Availability.model_validate(data)
    except FileNotFoundError:
        return None

def load_reviews_from_json(property_id: int):
    """
    Load all reviews for a property.
    """
    # Ensure directory exists
    Path(DATA_DIR).mkdir(parents=True, exist_ok=True)
    
    # List all review files
    review_files = list(Path(DATA_DIR).glob(f"review_*.json"))
    reviews = []
    
    for file_path in review_files:
        with open(file_path, 'r') as file:
            data = json.load(file)
            review = Review.model_validate(data)
            if review.property_id == property_id:
                reviews.append(review)
    
    return reviews

def load_all_properties():
    """
    Load all properties from JSON files.
    """
    # Ensure directory exists
    Path(DATA_DIR).mkdir(parents=True, exist_ok=True)
    
    # List all property files
    property_files = list(Path(DATA_DIR).glob("property_*.json"))
    properties = []
    
    for file_path in property_files:
        with open(file_path, 'r') as file:
            data = json.load(file)
            properties.append(Property.model_validate(data))
    
    return properties

# Generate sample data
def generate_sample_data():
    """
    Generate sample data for properties, availabilities, and reviews.
    """
    # Ensure directory exists
    Path(DATA_DIR).mkdir(parents=True, exist_ok=True)
    
    # Sample places
    places = ["New York", "London", "Paris", "Berlin", "Tokyo", "Sydney"]
    property_types = ["Apartment", "House", "Villa", "Loft", "Cottage"]
    
    # Generate properties
    for i in range(1, 21):  # 20 properties
        place = random.choice(places)
        property_type = random.choice(property_types)
        
        property = Property(
            id=i,
            name=f"Beautiful {property_type} in {place}",
            address=f"{random.randint(1, 100)} Main Street, {place}",
            title=f"Cozy {property_type} with great view",
            property_type=property_type,
            person_capacity=random.randint(1, 8),
            rating=round(random.uniform(3.5, 5.0), 1),
            latitude=random.uniform(-90, 90),
            longitude=random.uniform(-180, 180),
            price_per_night=random.randint(50, 300),
            place=place
        )
        save_object_to_json(property)
        
        # Generate availability for each property
        now = datetime.datetime.now()
        available_dates = []
        
        # Generate 15 random available dates within the next 30 days
        for _ in range(15):
            days_ahead = random.randint(1, 30)
            future_date = now + datetime.timedelta(days=days_ahead)
            date_str = future_date.strftime("%Y-%m-%d")
            if date_str not in available_dates:
                available_dates.append(date_str)
        
        availability = Availability(
            property_id=i,
            dates=sorted(available_dates)
        )
        save_object_to_json(availability)
        
        # Generate reviews for each property
        num_reviews = random.randint(1, 5)
        for j in range(num_reviews):
            days_ago = random.randint(1, 365)
            review_date = now - datetime.timedelta(days=days_ago)
            review = Review(
                property_id=i,
                user_name=f"User{random.randint(1, 100)}",
                comment=random.choice([
                    "Great place to stay!",
                    "Very clean and comfortable.",
                    "Nice location, close to everything.",
                    "Host was very helpful and responsive.",
                    "Would definitely recommend!",
                    "Had a wonderful time here."
                ]),
                rating=round(random.uniform(3.0, 5.0), 1),
                date=review_date.strftime("%Y-%m-%d")
            )
            save_object_to_json(review)

# API Functions
def search_property_by_place(place: str):
    """
    Search properties by place.
    
    Args:
        place (str): The name of the destination.
        
    Returns:
        list: A list of at most 3 properties, containing id, name, and address.
    """
    properties = load_all_properties()
    filtered_properties = [prop for prop in properties if prop.place.lower() == place.lower()]
    
    # Return at most 3 properties
    results = filtered_properties[:3]
    return {
        "property": [
            {"id": prop.id, "name": prop.name, "address": prop.address}
            for prop in results
        ]
    }

def get_property_details(propertyId: int):
    """
    Get property details by ID.
    
    Args:
        propertyId (int): The ID of the property.
        
    Returns:
        dict: Property details including title, type, capacity, rating, latitude, and longitude.
    """
    property = load_property_from_json(propertyId)
    if not property:
        return {"error": "Property not found"}
    
    return {
        "title": property.title,
        "propertyType": property.property_type,
        "personCapacity": property.person_capacity,
        "rating": property.rating,
        "latitude": property.latitude,
        "longitude": property.longitude
    }

def check_availability(propertyId: int):
    """
    Check property availability.
    
    Args:
        propertyId (int): The ID of the property.
        
    Returns:
        dict: The availability of the property for the next 10 days.
    """
    availability = load_availability_from_json(propertyId)
    if not availability:
        return {"availability": []}
    
    # Get today's date
    today = datetime.datetime.now()
    
    # Get availability for next 10 days
    next_10_days = []
    for i in range(10):
        date = today + datetime.timedelta(days=i)
        date_str = date.strftime("%Y-%m-%d")
        if date_str in availability.dates:
            next_10_days.append({"date": date_str, "available": True})
        else:
            next_10_days.append({"date": date_str, "available": False})
    
    return {"availability": next_10_days}

def get_property_reviews(propertyId: int):
    """
    Get property reviews.
    
    Args:
        propertyId (int): The ID of the property.
        
    Returns:
        dict: The reviews of the property (up to 3).
    """
    reviews = load_reviews_from_json(propertyId)
    
    # Return at most 3 reviews
    results = reviews[:3]
    return {
        "reviews": [
            {"comment": review.comment, "rating": review.rating, "date": review.date}
            for review in results
        ]
    }

def get_property_checkout_price(propertyId: int, checkIn: str, checkOut: str):
    """
    Get property checkout price.
    
    Args:
        propertyId (int): The ID of the property.
        checkIn (str): The check-in date in YYYY-MM-DD format.
        checkOut (str): The check-out date in YYYY-MM-DD format.
        
    Returns:
        dict: The accommodation cost.
    """
    property = load_property_from_json(propertyId)
    if not property:
        return {"error": "Property not found"}
    
    # Calculate number of nights
    try:
        check_in_date = datetime.datetime.strptime(checkIn, "%Y-%m-%d")
        check_out_date = datetime.datetime.strptime(checkOut, "%Y-%m-%d")
        
        if check_out_date <= check_in_date:
            return {"error": "Check-out date must be after check-in date"}
        
        nights = (check_out_date - check_in_date).days
        total_cost = property.price_per_night * nights
        
        return {"accommodationCost": round(total_cost, 2)}
    except ValueError:
        return {"error": "Invalid date format. Please use YYYY-MM-DD format."}

# Initialize sample data if needed
def initialize():
    # Check if data already exists
    if not list(Path(DATA_DIR).glob("property_*.json")):
        print("Generating sample data...")
        generate_sample_data()
        print("Sample data generated successfully!")

# Call initialize function when the module is imported
initialize()



Generating sample data...
Sample data generated successfully!


In [27]:
# Example usage of the Airbnb-like API functions

# 1. Search properties by place
def example_search_property():
    print("==== Searching properties in London ====")
    result = search_property_by_place("London")
    properties = result["property"]
    
    if properties:
        print(f"Found {len(properties)} properties:")
        for prop in properties:
            print(f"ID: {prop['id']}, Name: {prop['name']}")
            print(f"Address: {prop['address']}")
            print()
    else:
        print("No properties found in London")
    
    return properties[0]["id"] if properties else None

# 2. Get property details
def example_get_property_details(property_id):
    print(f"==== Getting details for property {property_id} ====")
    details = get_property_details(property_id)
    
    print(f"Title: {details['title']}")
    print(f"Type: {details['propertyType']}")
    print(f"Capacity: {details['personCapacity']} persons")
    print(f"Rating: {details['rating']}/5.0")
    print(f"Location: {details['latitude']}, {details['longitude']}")
    print()

# 3. Check availability
def example_check_availability(property_id):
    print(f"==== Checking availability for property {property_id} ====")
    result = check_availability(property_id)
    availability = result["availability"]
    
    print("Availability for the next 10 days:")
    for day in availability:
        status = "Available" if day.get("available", False) else "Not available"
        print(f"{day['date']}: {status}")
    print()
    
    # Return the first available date if any
    available_dates = [day["date"] for day in availability if day.get("available", False)]
    return available_dates[0] if available_dates else None

# 4. Get property reviews
def example_get_property_reviews(property_id):
    print(f"==== Getting reviews for property {property_id} ====")
    result = get_property_reviews(property_id)
    reviews = result["reviews"]
    
    if reviews:
        print(f"Found {len(reviews)} reviews:")
        for review in reviews:
            print(f"Rating: {review['rating']}/5.0 on {review['date']}")
            print(f"Comment: {review['comment']}")
            print()
    else:
        print("No reviews found for this property")

# 5. Get property checkout price
def example_get_property_checkout_price(property_id, available_date):
    if not available_date:
        print("No available dates to check pricing")
        return
    
    # Set checkout date to 3 days after check-in
    check_in = available_date
    check_out_date = datetime.datetime.strptime(check_in, "%Y-%m-%d") + datetime.timedelta(days=3)
    check_out = check_out_date.strftime("%Y-%m-%d")
    
    print(f"==== Getting price for property {property_id} ====")
    print(f"Check-in: {check_in}")
    print(f"Check-out: {check_out}")
    
    result = get_property_checkout_price(property_id, check_in, check_out)
    if "error" in result:
        print(f"Error: {result['error']}")
    else:
        print(f"Total accommodation cost: ${result['accommodationCost']}")
    print()

# Run a complete example workflow
def run_example_workflow():
    print("===================================")
    print("Running Airbnb API Example Workflow")
    print("===================================")
    
    # 1. Search for properties
    property_id = example_search_property()
    if not property_id:
        print("Cannot continue without a property")
        return
    
    # 2. Get details for the first property
    example_get_property_details(property_id)
    
    # 3. Check availability
    available_date = example_check_availability(property_id)
    
    # 4. Get reviews
    example_get_property_reviews(property_id)
    
    # 5. Get checkout price
    example_get_property_checkout_price(property_id, available_date)
    
    print("Workflow completed!")

# Run the example
if __name__ == "__main__":
    run_example_workflow()



Running Airbnb API Example Workflow
==== Searching properties in London ====
Found 2 properties:
ID: 13, Name: Beautiful Villa in London
Address: 75 Main Street, London

ID: 10, Name: Beautiful Cottage in London
Address: 18 Main Street, London

==== Getting details for property 13 ====
Title: Cozy Villa with great view
Type: Villa
Capacity: 1 persons
Rating: 3.7/5.0
Location: 1.5359839571144676, -124.10800590408309

==== Checking availability for property 13 ====
Availability for the next 10 days:
2025-02-26: Not available
2025-02-27: Not available
2025-02-28: Not available
2025-03-01: Not available
2025-03-02: Not available
2025-03-03: Available
2025-03-04: Not available
2025-03-05: Available
2025-03-06: Not available
2025-03-07: Not available

==== Getting reviews for property 13 ====
Found 2 reviews:
Rating: 4.5/5.0 on 2025-02-17
Comment: Had a wonderful time here.

Rating: 3.1/5.0 on 2025-02-24
Comment: Had a wonderful time here.

==== Getting price for property 13 ====
Check-in: 2

In [28]:
import uuid
import json
import random
import math
from uuid import UUID, uuid4
from typing import List, Dict, Any, Optional
from pydantic import BaseModel, Field
from pathlib import Path

# Define base models for the map system
class Location(BaseModel):
    id: UUID = Field(default_factory=uuid4)
    name: str
    address: str
    city: str
    state: str
    latitude: float
    longitude: float
    
    def model_post_init(self, __context):
        # Auto-save object after initialization
        save_object_to_json(self)

class Route(BaseModel):
    id: UUID = Field(default_factory=uuid4)
    start_location: str
    end_location: str
    distance: float  # in kilometers
    duration: float  # in minutes
    steps: List[str]  # list of navigation instructions
    
    def model_post_init(self, __context):
        # Auto-save object after initialization
        save_object_to_json(self)

# Directory to store json files
DATA_DIR = "./map_data"

def save_object_to_json(obj: BaseModel):
    """
    Save the given object as a JSON file.
    """
    # Ensure directory exists
    Path(DATA_DIR).mkdir(parents=True, exist_ok=True)
    
    # Use object's id as filename
    if isinstance(obj, Location):
        file_path = Path(DATA_DIR) / f"location_{obj.id}.json"
    elif isinstance(obj, Route):
        file_path = Path(DATA_DIR) / f"route_{obj.id}.json"
    else:
        file_path = Path(DATA_DIR) / f"{obj.id}.json"
    
    # Serialize to JSON string
    json_str = obj.model_dump_json()
    
    # Write to file
    with open(file_path, 'w') as file:
        file.write(json_str)

def load_location_by_name(location_name: str) -> Optional[Location]:
    """
    Load a location by name from JSON files.
    """
    # Ensure directory exists
    Path(DATA_DIR).mkdir(parents=True, exist_ok=True)
    
    # List all location files
    location_files = list(Path(DATA_DIR).glob("location_*.json"))
    
    for file_path in location_files:
        with open(file_path, 'r') as file:
            data = json.load(file)
            location = Location.model_validate(data)
            if location.name.lower() == location_name.lower() or location_name.lower() in location.address.lower():
                return location
    
    return None

def load_all_locations() -> List[Location]:
    """
    Load all locations from JSON files.
    """
    # Ensure directory exists
    Path(DATA_DIR).mkdir(parents=True, exist_ok=True)
    
    # List all location files
    location_files = list(Path(DATA_DIR).glob("location_*.json"))
    locations = []
    
    for file_path in location_files:
        with open(file_path, 'r') as file:
            data = json.load(file)
            locations.append(Location.model_validate(data))
    
    return locations

def load_route(start: str, end: str) -> Optional[Route]:
    """
    Load a route by start and end locations from JSON files.
    """
    # Ensure directory exists
    Path(DATA_DIR).mkdir(parents=True, exist_ok=True)
    
    # List all route files
    route_files = list(Path(DATA_DIR).glob("route_*.json"))
    
    for file_path in route_files:
        with open(file_path, 'r') as file:
            data = json.load(file)
            route = Route.model_validate(data)
            if (route.start_location.lower() == start.lower() and 
                route.end_location.lower() == end.lower()):
                return route
    
    return None

def haversine_distance(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371  # Radius of earth in kilometers
    return c * r

def generate_route_steps(start: Location, end: Location) -> List[str]:
    """
    Generate a list of navigation steps between two locations.
    """
    # This is a simplified version that generates random steps
    directions = ["north", "northeast", "east", "southeast", "south", "southwest", "west", "northwest"]
    road_types = ["Avenue", "Boulevard", "Street", "Road", "Drive", "Lane", "Way", "Place"]
    
    steps = []
    
    # Starting step
    steps.append(f"Head {random.choice(directions)} on {start.address}")
    
    # Generate 3-5 intermediate steps
    num_steps = random.randint(3, 5)
    for _ in range(num_steps):
        action = random.choice(["Turn left", "Turn right", "Continue", "Merge", "Take the exit"])
        road_name = f"{random.choice(['Main', 'Oak', 'Pine', 'Maple', 'Cedar', 'Elm'])} {random.choice(road_types)}"
        distance = round(random.uniform(0.5, 5.0), 1)
        steps.append(f"{action} onto {road_name} for {distance} km")
    
    # Final step
    steps.append(f"Arrive at {end.address}")
    
    return steps

# Generate sample data
def generate_sample_data():
    """
    Generate sample data for locations and routes.
    """
    # Ensure directory exists
    Path(DATA_DIR).mkdir(parents=True, exist_ok=True)
    
    # Sample US cities with coordinates
    cities = [
        {"name": "Empire State Building", "address": "350 Fifth Avenue", "city": "New York", "state": "NY", "latitude": 40.7484, "longitude": -73.9857},
        {"name": "Golden Gate Bridge", "address": "Golden Gate Bridge", "city": "San Francisco", "state": "CA", "latitude": 37.8199, "longitude": -122.4783},
        {"name": "Space Needle", "address": "400 Broad St", "city": "Seattle", "state": "WA", "latitude": 47.6205, "longitude": -122.3493},
        {"name": "Willis Tower", "address": "233 S Wacker Dr", "city": "Chicago", "state": "IL", "latitude": 41.8789, "longitude": -87.6359},
        {"name": "Gateway Arch", "address": "11 N 4th St", "city": "St. Louis", "state": "MO", "latitude": 38.6247, "longitude": -90.1848},
        {"name": "Liberty Bell", "address": "526 Market St", "city": "Philadelphia", "state": "PA", "latitude": 39.9496, "longitude": -75.1503},
        {"name": "Hollywood Sign", "address": "Hollywood Sign", "city": "Los Angeles", "state": "CA", "latitude": 34.1341, "longitude": -118.3215},
        {"name": "White House", "address": "1600 Pennsylvania Avenue NW", "city": "Washington", "state": "DC", "latitude": 38.8977, "longitude": -77.0365},
        {"name": "French Quarter", "address": "700 Decatur St", "city": "New Orleans", "state": "LA", "latitude": 29.9584, "longitude": -90.0644},
        {"name": "Times Square", "address": "Manhattan", "city": "New York", "state": "NY", "latitude": 40.7580, "longitude": -73.9855},
        {"name": "Grand Canyon", "address": "Grand Canyon", "city": "Grand Canyon Village", "state": "AZ", "latitude": 36.0544, "longitude": -112.1401},
        {"name": "Mount Rushmore", "address": "13000 SD-244", "city": "Keystone", "state": "SD", "latitude": 43.8791, "longitude": -103.4591},
        {"name": "Kennedy Space Center", "address": "Space Commerce Way", "city": "Merritt Island", "state": "FL", "latitude": 28.5728, "longitude": -80.6490},
        {"name": "Alamo", "address": "300 Alamo Plaza", "city": "San Antonio", "state": "TX", "latitude": 29.4260, "longitude": -98.4861},
        {"name": "Statue of Liberty", "address": "Liberty Island", "city": "New York", "state": "NY", "latitude": 40.6892, "longitude": -74.0445},
        {"name": "Mall of America", "address": "60 E Broadway", "city": "Bloomington", "state": "MN", "latitude": 44.8549, "longitude": -93.2422},
        {"name": "Yellowstone National Park", "address": "Yellowstone National Park", "city": "Yellowstone", "state": "WY", "latitude": 44.4280, "longitude": -110.5885},
        {"name": "Las Vegas Strip", "address": "Las Vegas Blvd", "city": "Las Vegas", "state": "NV", "latitude": 36.1147, "longitude": -115.1728},
        {"name": "Niagara Falls", "address": "Niagara Falls", "city": "Niagara Falls", "state": "NY", "latitude": 43.0962, "longitude": -79.0377},
        {"name": "Disneyland", "address": "1313 Disneyland Dr", "city": "Anaheim", "state": "CA", "latitude": 33.8121, "longitude": -117.9190}
    ]
    
    # Generate locations
    locations = []
    for city_data in cities:
        location = Location(
            name=city_data["name"],
            address=f"{city_data['address']}, {city_data['city']}, {city_data['state']}",
            city=city_data["city"],
            state=city_data["state"],
            latitude=city_data["latitude"],
            longitude=city_data["longitude"]
        )
        locations.append(location)
        save_object_to_json(location)
    
    # Generate some routes between random pairs of locations
    for _ in range(15):  # Generate 15 random routes
        start_loc = random.choice(locations)
        end_loc = random.choice([loc for loc in locations if loc.id != start_loc.id])
        
        distance = haversine_distance(
            start_loc.latitude, start_loc.longitude, 
            end_loc.latitude, end_loc.longitude
        )
        
        # Average speed 60 km/h -> calculate duration in minutes
        duration = (distance / 60) * 60
        
        steps = generate_route_steps(start_loc, end_loc)
        
        route = Route(
            start_location=start_loc.name,
            end_location=end_loc.name,
            distance=round(distance, 2),
            duration=round(duration, 2),
            steps=steps
        )
        save_object_to_json(route)

# API Functions
def get_distance(start: str, end: str) -> Dict[str, Any]:
    """
    Get the distance between two locations.
    
    Args:
        start (str): The start location.
        end (str): The end location.
        
    Returns:
        dict: The distance in km.
    """
    # Try to find the route in existing data
    route = load_route(start, end)
    
    if route:
        return {"distance": route.distance}
    
    # If route not found, try to compute distance from locations
    start_loc = load_location_by_name(start)
    end_loc = load_location_by_name(end)
    
    if start_loc and end_loc:
        distance = haversine_distance(
            start_loc.latitude, start_loc.longitude,
            end_loc.latitude, end_loc.longitude
        )
        return {"distance": round(distance, 2)}
    
    # If locations not found, return error
    return {"error": "Could not find one or both locations."}

def get_route(start: str, end: str) -> Dict[str, Any]:
    """
    Get the route between two locations.
    
    Args:
        start (str): The start location.
        end (str): The end location.
        
    Returns:
        dict: The route, a list of actions.
    """
    # Try to find the route in existing data
    route = load_route(start, end)
    
    if route:
        return {"route": route.steps}
    
    # If route not found, try to generate one from locations
    start_loc = load_location_by_name(start)
    end_loc = load_location_by_name(end)
    
    if start_loc and end_loc:
        steps = generate_route_steps(start_loc, end_loc)
        
        # Save the new route
        distance = haversine_distance(
            start_loc.latitude, start_loc.longitude,
            end_loc.latitude, end_loc.longitude
        )
        duration = (distance / 60) * 60
        
        new_route = Route(
            start_location=start_loc.name,
            end_location=end_loc.name,
            distance=round(distance, 2),
            duration=round(duration, 2),
            steps=steps
        )
        save_object_to_json(new_route)
        
        return {"route": steps}
    
    # If locations not found, return error
    return {"error": "Could not find one or both locations."}

def get_coordinates(location: str) -> Dict[str, Any]:
    """
    Get the coordinates of a location.
    
    Args:
        location (str): The location need to get coordinates.
        
    Returns:
        dict: The latitude and longitude of the location.
    """
    loc = load_location_by_name(location)
    
    if loc:
        return {
            "latitude": loc.latitude,
            "longitude": loc.longitude
        }
    
    # If location not found, return error
    return {"error": "Location not found."}

def search_nearby(search_term: str) -> Dict[str, Any]:
    """
    Search for places by name.
    
    Args:
        search_term (str): The place name.
        
    Returns:
        dict: The list of places, each place is a dict with name and address, at most 5 places.
    """
    all_locations = load_all_locations()
    
    # Filter locations by search term (name or address contains the term)
    matching_locations = [
        loc for loc in all_locations 
        if search_term.lower() in loc.name.lower() or 
           search_term.lower() in loc.address.lower() or
           search_term.lower() in loc.city.lower() or
           search_term.lower() in loc.state.lower()
    ]
    
    # Return at most 5 places
    results = matching_locations[:5]
    return {
        "places": [
            {"name": loc.name, "address": loc.address}
            for loc in results
        ]
    }

# Initialize sample data if needed
def initialize():
    # Check if data already exists
    if not list(Path(DATA_DIR).glob("location_*.json")):
        print("Generating sample map data...")
        generate_sample_data()
        print("Sample map data generated successfully!")

# Call initialize function when the module is imported
initialize()

# Example usage of the map API functions
def example_usage():
    print("===== Map Information Lookup API Examples =====")
    
    # 1. Get coordinates
    location = "Empire State Building"
    print(f"\n1. Getting coordinates for {location}")
    coords = get_coordinates(location)
    if "error" in coords:
        print(f"Error: {coords['error']}")
    else:
        print(f"Latitude: {coords['latitude']}")
        print(f"Longitude: {coords['longitude']}")
    
    # 2. Search nearby
    search = "New York"
    print(f"\n2. Searching for places with term '{search}'")
    results = search_nearby(search)
    if "places" in results and results["places"]:
        print(f"Found {len(results['places'])} places:")
        for place in results["places"]:
            print(f"- {place['name']}: {place['address']}")
    else:
        print(f"No places found for '{search}'")
    
    # 3. Get distance
    start = "Empire State Building"
    end = "Statue of Liberty"
    print(f"\n3. Getting distance from {start} to {end}")
    distance_result = get_distance(start, end)
    if "error" in distance_result:
        print(f"Error: {distance_result['error']}")
    else:
        print(f"Distance: {distance_result['distance']} km")
    
    # 4. Get route
    print(f"\n4. Getting route from {start} to {end}")
    route_result = get_route(start, end)
    if "error" in route_result:
        print(f"Error: {route_result['error']}")
    else:
        print("Route steps:")
        for i, step in enumerate(route_result["route"]):
            print(f"  Step {i+1}: {step}")

# Run example if script is executed directly
if __name__ == "__main__":
    example_usage()



Generating sample map data...
Sample map data generated successfully!
===== Map Information Lookup API Examples =====

1. Getting coordinates for Empire State Building
Latitude: 40.7484
Longitude: -73.9857

2. Searching for places with term 'New York'
Found 3 places:
- Statue of Liberty: Liberty Island, New York, NY
- Empire State Building: 350 Fifth Avenue, New York, NY
- Times Square: Manhattan, New York, NY

3. Getting distance from Empire State Building to Statue of Liberty
Distance: 8.24 km

4. Getting route from Empire State Building to Statue of Liberty
Route steps:
  Step 1: Head west on 350 Fifth Avenue, New York, NY
  Step 2: Continue onto Pine Place for 2.9 km
  Step 3: Turn left onto Main Road for 4.8 km
  Step 4: Continue onto Main Boulevard for 4.2 km
  Step 5: Turn left onto Cedar Street for 4.0 km
  Step 6: Arrive at Liberty Island, New York, NY


In [32]:
import uuid
import json
import random
from uuid import UUID, uuid4
from typing import List, Dict, Any, Optional
from pydantic import BaseModel, Field
from pathlib import Path

# Define base models for the movie rating system
class Movie(BaseModel):
    id: UUID = Field(default_factory=uuid4)
    title: str
    genre: str
    director: str
    release_year: int
    rating: float  # Rating out of 10
    runtime: int   # Runtime in minutes
    
    def model_post_init(self, __context):
        # Auto-save object after initialization
        save_object_to_json(self)

class Review(BaseModel):
    id: UUID = Field(default_factory=uuid4)
    movie_title: str
    reviewer: str
    score: float  # Score out of 10
    comment: str
    
    def model_post_init(self, __context):
        # Auto-save object after initialization
        save_object_to_json(self)

# Directory to store json files
DATA_DIR = "./movie_data"

def save_object_to_json(obj: BaseModel):
    """
    Save the given object as a JSON file.
    """
    # Ensure directory exists
    Path(DATA_DIR).mkdir(parents=True, exist_ok=True)
    
    # Use object's id as filename
    if isinstance(obj, Movie):
        file_path = Path(DATA_DIR) / f"movie_{obj.id}.json"
    elif isinstance(obj, Review):
        file_path = Path(DATA_DIR) / f"review_{obj.id}.json"
    else:
        file_path = Path(DATA_DIR) / f"{obj.id}.json"
    
    # Serialize to JSON string
    json_str = obj.model_dump_json()
    
    # Write to file
    with open(file_path, 'w') as file:
        file.write(json_str)

def load_movie_by_title(movie_title: str) -> Optional[Movie]:
    """
    Load a movie by title from JSON files.
    """
    Path(DATA_DIR).mkdir(parents=True, exist_ok=True)
    
    movie_files = list(Path(DATA_DIR).glob("movie_*.json"))
    
    for file_path in movie_files:
        with open(file_path, 'r') as file:
            data = json.load(file)
            movie = Movie.model_validate(data)
            if movie.title.lower() == movie_title.lower():
                return movie
    
    return None

def load_all_movies() -> List[Movie]:
    """
    Load all movies from JSON files.
    """
    Path(DATA_DIR).mkdir(parents=True, exist_ok=True)
    
    movie_files = list(Path(DATA_DIR).glob("movie_*.json"))
    movies = []
    
    for file_path in movie_files:
        with open(file_path, 'r') as file:
            data = json.load(file)
            movies.append(Movie.model_validate(data))
    
    return movies

def load_reviews_by_movie(movie_title: str) -> List[Review]:
    """
    Load all reviews for a specific movie from JSON files.
    """
    Path(DATA_DIR).mkdir(parents=True, exist_ok=True)
    
    review_files = list(Path(DATA_DIR).glob("review_*.json"))
    reviews = []
    
    for file_path in review_files:
        with open(file_path, 'r') as file:
            data = json.load(file)
            review = Review.model_validate(data)
            if review.movie_title.lower() == movie_title.lower():
                reviews.append(review)
    
    return reviews

# Generate sample data
def generate_sample_data():
    """
    Generate sample data for movies and reviews.
    """
    Path(DATA_DIR).mkdir(parents=True, exist_ok=True)
    
    # Sample movie data
    movies_data = [
        {"title": "The Shawshank Redemption", "genre": "Drama", "director": "Frank Darabont", "release_year": 1994, "rating": 9.3, "runtime": 142},
        {"title": "The Godfather", "genre": "Crime", "director": "Francis Ford Coppola", "release_year": 1972, "rating": 9.2, "runtime": 175},
        {"title": "Pulp Fiction", "genre": "Crime", "director": "Quentin Tarantino", "release_year": 1994, "rating": 8.9, "runtime": 154},
        {"title": "Inception", "genre": "Sci-Fi", "director": "Christopher Nolan", "release_year": 2010, "rating": 8.8, "runtime": 148},
        {"title": "The Dark Knight", "genre": "Action", "director": "Christopher Nolan", "release_year": 2008, "rating": 9.0, "runtime": 152},
        {"title": "Forrest Gump", "genre": "Drama", "director": "Robert Zemeckis", "release_year": 1994, "rating": 8.8, "runtime": 142},
        {"title": "The Matrix", "genre": "Sci-Fi", "director": "Lana Wachowski", "release_year": 1999, "rating": 8.7, "runtime": 136},
        {"title": "Titanic", "genre": "Romance", "director": "James Cameron", "release_year": 1997, "rating": 7.8, "runtime": 195},
        {"title": "Avatar", "genre": "Sci-Fi", "director": "James Cameron", "release_year": 2009, "rating": 7.8, "runtime": 162},
        {"title": "Jurassic Park", "genre": "Adventure", "director": "Steven Spielberg", "release_year": 1993, "rating": 8.1, "runtime": 127},
    ]
    
    # Generate movies
    movies = []
    for movie_data in movies_data:
        movie = Movie(
            title=movie_data["title"],
            genre=movie_data["genre"],
            director=movie_data["director"],
            release_year=movie_data["release_year"],
            rating=movie_data["rating"],
            runtime=movie_data["runtime"]
        )
        movies.append(movie)
        save_object_to_json(movie)
    
    # Generate reviews for each movie
    reviewers = ["Alice", "Bob", "Charlie", "Diana", "Eve"]
    comments = [
        "Amazing film!", "A masterpiece.", "Really enjoyed it.", 
        "Could have been better.", "Mind-blowing visuals!"
    ]
    
    for movie in movies:
        num_reviews = random.randint(1, 5)
        for _ in range(num_reviews):
            review = Review(
                movie_title=movie.title,
                reviewer=random.choice(reviewers),
                score=round(random.uniform(max(1.0, movie.rating - 2), min(10.0, movie.rating + 1)), 1),
                comment=random.choice(comments)
            )
            save_object_to_json(review)

# API Functions
def get_movie_info(movie_title: str) -> Dict[str, Any]:
    """
    Get detailed information about a movie.
    
    Args:
        movie_title (str): The title of the movie.
        
    Returns:
        dict: Movie details including title, genre, director, release year, rating, and runtime.
    """
    movie = load_movie_by_title(movie_title)
    
    if movie:
        return {
            "title": movie.title,
            "genre": movie.genre,
            "director": movie.director,
            "release_year": movie.release_year,
            "rating": movie.rating,
            "runtime": movie.runtime
        }
    
    return {"error": "Movie not found."}

def get_movie_rating(movie_title: str) -> Dict[str, Any]:
    """
    Get the average rating of a movie based on critic score and user reviews.
    
    Args:
        movie_title (str): The title of the movie.
        
    Returns:
        dict: The average rating.
    """
    movie = load_movie_by_title(movie_title)
    reviews = load_reviews_by_movie(movie_title)
    
    if movie:
        ratings = [movie.rating] + [review.score for review in reviews]
        avg_rating = round(sum(ratings) / len(ratings), 1) if ratings else movie.rating
        return {"rating": avg_rating}
    
    return {"error": "Movie not found."}

def get_movie_reviews(movie_title: str) -> Dict[str, Any]:
    """
    Get all reviews for a movie.
    
    Args:
        movie_title (str): The title of the movie.
        
    Returns:
        dict: A list of reviews.
    """
    reviews = load_reviews_by_movie(movie_title)
    
    if reviews:
        return {
            "reviews": [
                {"reviewer": review.reviewer, "score": review.score, "comment": review.comment}
                for review in reviews
            ]
        }
    
    return {"error": "No reviews found for this movie."}

def search_movies(search_term: str) -> Dict[str, Any]:
    """
    Search for movies by title, genre, or director.
    
    Args:
        search_term (str): The search term.
        
    Returns:
        dict: A list of matching movies, up to 5 results.
    """
    all_movies = load_all_movies()
    
    # Filter movies by search term
    matching_movies = [
        movie for movie in all_movies 
        if (search_term.lower() in movie.title.lower() or 
            search_term.lower() in movie.genre.lower() or 
            search_term.lower() in movie.director.lower())
    ]
    
    # Return at most 5 results
    results = matching_movies[:5]
    return {
        "movies": [
            {"title": movie.title, "genre": movie.genre, "rating": movie.rating}
            for movie in results
        ]
    }

# Initialize sample data if needed
def initialize():
    if not list(Path(DATA_DIR).glob("movie_*.json")):
        print("Generating sample movie data...")
        generate_sample_data()
        print("Sample movie data generated successfully!")

# Call initialize function when the module is imported
initialize()

# Example usage of the movie API functions
def example_usage():
    print("===== Movie Rating Lookup API Examples =====")
    
    # 1. Get movie info
    movie = "Inception"
    print(f"\n1. Getting info for {movie}")
    info = get_movie_info(movie)
    if "error" in info:
        print(f"Error: {info['error']}")
    else:
        print(f"Title: {info['title']}")
        print(f"Genre: {info['genre']}")
        print(f"Director: {info['director']}")
        print(f"Release Year: {info['release_year']}")
        print(f"Rating: {info['rating']}")
        print(f"Runtime: {info['runtime']} minutes")
    
    # 2. Get movie rating
    print(f"\n2. Getting average rating for {movie}")
    rating = get_movie_rating(movie)
    if "error" in rating:
        print(f"Error: {rating['error']}")
    else:
        print(f"Average Rating: {rating['rating']}")
    
    # 3. Get movie reviews
    print(f"\n3. Getting reviews for {movie}")
    reviews = get_movie_reviews(movie)
    if "error" in reviews:
        print(f"Error: {reviews['error']}")
    else:
        print(f"Found {len(reviews['reviews'])} reviews:")
        for review in reviews["reviews"]:
            print(f"- {review['reviewer']}: {review['score']}/10 - {review['comment']}")
    
    # 4. Search movies
    search = "Nolan"
    print(f"\n4. Searching for movies with term '{search}'")
    results = search_movies(search)
    if "movies" in results and results["movies"]:
        print(f"Found {len(results['movies'])} movies:")
        for movie in results["movies"]:
            print(f"- {movie['title']} ({movie['genre']}): {movie['rating']}/10")
    else:
        print(f"No movies found for '{search}'")

# Run example if script is executed directly
if __name__ == "__main__":
    example_usage()


Generating sample movie data...
Sample movie data generated successfully!
===== Movie Rating Lookup API Examples =====

1. Getting info for Inception
Title: Inception
Genre: Sci-Fi
Director: Christopher Nolan
Release Year: 2010
Rating: 8.8
Runtime: 148 minutes

2. Getting average rating for Inception
Average Rating: 8.5

3. Getting reviews for Inception
Found 5 reviews:
- Eve: 8.1/10 - Could have been better.
- Diana: 9.2/10 - Really enjoyed it.
- Charlie: 7.7/10 - Mind-blowing visuals!
- Diana: 9.5/10 - Mind-blowing visuals!
- Diana: 7.6/10 - Mind-blowing visuals!

4. Searching for movies with term 'Nolan'
Found 2 movies:
- The Dark Knight (Action): 9.0/10
- Inception (Sci-Fi): 8.8/10
